In [1]:
import gym
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1}
    cont_env = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, 
                     learning_rate = learning_rate,use_sde = False, gae_lambda = gae, n_steps = n_step)
    env_eval = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=60000)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 32767+32767

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        print(res_mean_arr)
    
#     plt.pyplot.plot(res_mean_arr)

    return res_mean_arr, res_std_arr

In [ ]:
listp = [0.25,1,4,9,39,99]
listL = [70,100]
# listp = [99]
# listL = [100]
t_t = 2000000
n_iter = 10000 #for eval
n_step = 8192
learning_rate = 0.0003
gae_lambda = 0.99
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        res_mean, res_std = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':min(res_mean), 'res_std': res_std[np.argmin(res_mean)]}, ignore_index=True)

Running PPO w/: p= 0.25 , L= 70
Using cpu device
------------------------------
| time/              |       |
|    fps             | 5296  |
|    iterations      | 1     |
|    time_elapsed    | 6     |
|    total_timesteps | 32768 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2517        |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.009051868 |
|    clip_fraction        | 0.0417      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.99       |
|    explained_variance   | -0.000243   |
|    learning_rate        | 0.0003      |
|    loss                 | 8.46e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.003      |
|    value_loss           | 2.79e+06    |
--------------------

-----------------------------------------
| time/                   |             |
|    fps                  | 2170        |
|    iterations           | 2           |
|    time_elapsed         | 30          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.017520554 |
|    clip_fraction        | 0.071       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.75       |
|    explained_variance   | 1.29e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 315         |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00657    |
|    value_loss           | 4.33e+03    |
-----------------------------------------
mean:  -0.8470883182743492
standard deviation: 0.0310662277868289
[3.919922970210827, 1.8030880331934882, 1.3589689237795624, 1.1743728900212962, 0.9370294616272222, 0.8470883182743492]
-----------------------------------------
| time/         

-----------------------------------------
| time/                   |             |
|    fps                  | 2282        |
|    iterations           | 2           |
|    time_elapsed         | 28          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.007798563 |
|    clip_fraction        | 0.12        |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.51       |
|    explained_variance   | 0.0379      |
|    learning_rate        | 0.0003      |
|    loss                 | 48.5        |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.0131     |
|    value_loss           | 236         |
-----------------------------------------
mean:  -0.4751259794801542
standard deviation: 0.006087353387019741
[3.919922970210827, 1.8030880331934882, 1.3589689237795624, 1.1743728900212962, 0.9370294616272222, 0.8470883182743492, 0.7567717448942096, 0.6705708905536654, 0.60319729993353

-----------------------------------------
| time/                   |             |
|    fps                  | 5750        |
|    iterations           | 1           |
|    time_elapsed         | 5           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.010588866 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.21       |
|    explained_variance   | 0.166       |
|    learning_rate        | 0.0003      |
|    loss                 | 14.4        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0123     |
|    value_loss           | 62.7        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2654        |
|    iterations           | 2           |
|    time_elapsed         | 24          |
|    total_timesteps      | 65536 

-----------------------------------------
| time/                   |             |
|    fps                  | 2527        |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.009858831 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | 0.126       |
|    learning_rate        | 0.0003      |
|    loss                 | 9.08        |
|    n_updates            | 390         |
|    policy_gradient_loss | -0.0111     |
|    value_loss           | 35.2        |
-----------------------------------------
mean:  -0.28079707844684904
standard deviation: 0.0012252789613316764
[3.919922970210827, 1.8030880331934882, 1.3589689237795624, 1.1743728900212962, 0.9370294616272222, 0.8470883182743492, 0.7567717448942096, 0.6705708905536654, 0.603197299933

...

-----------------------------------------
| time/                   |             |
|    fps                  | 2490        |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.014982999 |
|    clip_fraction        | 0.034       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.47       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 120         |
|    n_updates            | 230         |
|    policy_gradient_loss | -0.00248    |
|    value_loss           | 3.21e+03    |
-----------------------------------------
mean:  -0.9680859459861239
standard deviation: 0.029872074018750947
[3.2210449356879316, 1.9229426409276478, 1.4535880694196508, 1.2189529649193656, 1.1602342359012394, 1.1061211672357125, 1.0665790046206354, 1.0646301669670604, 1.0397918343380

-----------------------------------------
| time/                   |             |
|    fps                  | 5767        |
|    iterations           | 1           |
|    time_elapsed         | 5           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.014190445 |
|    clip_fraction        | 0.0426      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.35       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 41.4        |
|    n_updates            | 320         |
|    policy_gradient_loss | -0.00189    |
|    value_loss           | 2.45e+03    |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2452        |
|    iterations           | 2           |
|    time_elapsed         | 26          |
|    total_timesteps      | 65536 

-----------------------------------------
| time/                   |             |
|    fps                  | 2545        |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.009997267 |
|    clip_fraction        | 0.0386      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.18       |
|    explained_variance   | -3.58e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 64.4        |
|    n_updates            | 410         |
|    policy_gradient_loss | -0.00163    |
|    value_loss           | 1.95e+03    |
-----------------------------------------
mean:  -0.8224148915129772
standard deviation: 0.008925435636416396
[3.2210449356879316, 1.9229426409276478, 1.4535880694196508, 1.2189529649193656, 1.1602342359012394, 1.1061211672357125, 1.0665790046206354, 1.0646301669670604, 1.0397918343380

mean:  -0.8132133583466776
standard deviation: 0.009666917272752991
[3.2210449356879316, 1.9229426409276478, 1.4535880694196508, 1.2189529649193656, 1.1602342359012394, 1.1061211672357125, 1.0665790046206354, 1.0646301669670604, 1.039791834338087, 1.0163669158360422, 1.002039286655679, 0.9680859459861239, 0.9461677082311353, 0.9478653133119426, 0.9276373707447718, 0.8756474082500301, 0.8577672707550542, 0.8457955018252312, 0.8350758865844801, 0.8363432090519265, 0.8224148915129772, 0.8171554013807402, 0.8207823101855777, 0.8177512101101296, 0.8132133583466776]
-----------------------------------------
| time/                   |             |
|    fps                  | 5634        |
|    iterations           | 1           |
|    time_elapsed         | 5           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.010239974 |
|    clip_fraction        | 0.038       |
|    clip_range           | 0.2         |
|    entropy_

mean:  -0.807772466133019
standard deviation: 0.003116341324503846
[3.2210449356879316, 1.9229426409276478, 1.4535880694196508, 1.2189529649193656, 1.1602342359012394, 1.1061211672357125, 1.0665790046206354, 1.0646301669670604, 1.039791834338087, 1.0163669158360422, 1.002039286655679, 0.9680859459861239, 0.9461677082311353, 0.9478653133119426, 0.9276373707447718, 0.8756474082500301, 0.8577672707550542, 0.8457955018252312, 0.8350758865844801, 0.8363432090519265, 0.8224148915129772, 0.8171554013807402, 0.8207823101855777, 0.8177512101101296, 0.8132133583466776, 0.8168790043243058, 0.8121054361901434, 0.8115750844605124, 0.807772466133019]
-----------------------------------------
| time/                   |             |
|    fps                  | 3370        |
|    iterations           | 1           |
|    time_elapsed         | 9           |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.010054562 |
|    clip_fraction 

In [ ]:
ppo_res

In [ ]:
plt.pyplot.plot(res_mean)